<a href="https://colab.research.google.com/github/BrockDSL/antiwork_reddit_data_set/blob/main/antiwork_data_set_first_pass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# r/antiwork first pass of data

headers etc grabbed with [https://github.com/Jabb0/SubredditDownloader](https://github.com/Jabb0/SubredditDownloader)

info is kept in an SQLite database in [github repo](https://github.com/BrockDSL/antiwork_reddit_data_set)

This does inital grab of data and clean-up

In [150]:
import pandas as pd
import requests
import sqlite3

from datetime import datetime
from dateutil.tz import tzutc, tzlocal
from google.colab import files

print("Libraries loaded")


#This cell will open the sqlite file load and process it
#it will take at least a minute for this cell to complete


#grab file and make connection
!wget -O anti-work.db https://github.com/BrockDSL/antiwork_reddit_data_set/blob/main/anti-work.db?raw=true

con = sqlite3.connect("anti-work.db")
adf = pd.read_sql_query("SELECT * FROM submissions",con)


#Un-needed columns
del(adf['downloaded'])
del(adf['download_file'])
del(adf['downloaded_at'])

#Drop Jan 1 posts
adf = adf[adf['created_utc'] < 1640995200]
adf = adf.reset_index(drop=True)

print("Full length of dataset: ",len(adf))
print("Number of deleted posts: ", len(adf[ adf['author'] == '[deleted]']))

#Drop deleted posts
adf = adf.drop(adf[ adf['author'] == '[deleted]'].index)
adf = adf.reset_index(drop=True)
print("New length of dataset: ",len(adf))

#Converting UTC to timestmp
dates = []
for row in adf.itertuples(index=False):
  dates.append(pd.to_datetime(row.created_utc,unit='s'))

dates_df = pd.DataFrame(dates,columns = ["timestamp"])
adf = adf.join(dates_df)

#Link to post

post_url = []

for row in adf.itertuples(index=False):
  post_url.append("https://reddit.com/"+row.id)

post_url = pd.DataFrame(post_url,columns = ["post_url"])
adf = adf.join(post_url)

#redo columns
adf = adf.reindex(columns=['timestamp', 'post_url','id','author','title','url','upvote_ratio','score','removed_by_category','created_utc'])

#sort oldest to newest
adf = adf.sort_values(by=['created_utc'],ignore_index=True)


#Download final dataset
adf.to_csv("antiwork.csv",index=False)
files.download("antiwork.csv")


print("Dataset Constructed")

--2022-03-04 15:12:42--  https://github.com/BrockDSL/antiwork_reddit_data_set/blob/main/anti-work.db?raw=true
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/BrockDSL/antiwork_reddit_data_set/raw/main/anti-work.db [following]
--2022-03-04 15:12:42--  https://github.com/BrockDSL/antiwork_reddit_data_set/raw/main/anti-work.db
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BrockDSL/antiwork_reddit_data_set/main/anti-work.db [following]
--2022-03-04 15:12:42--  https://raw.githubusercontent.com/BrockDSL/antiwork_reddit_data_set/main/anti-work.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|: